### DQN 구현

- 풀고자 하는 문제인 카트폴(CartPole)은 카트를 잘 밀어서 막대가 넘어지지 않도록 균형을 잡는 문제
- 카트는 일정한 힘으로 왼쪽이나 오른쪽으로 밀 수 있기 때문에 선택할 수 있는 액션은 항상 2가지뿐, 또한 스텝마다 +1의 보상을 받기 때문에 보상을 최적화하는 것은 곧 막대를 넘어뜨리지 않고 가능한 오래도록 균형을 잡는 것
- 막대가 수직으로부터 15도 이상 기울어지거나 카트가 화면 끝으로 나가면 종료
- 카트의 상태 s는 길이 4의 벡터 s = (카트의 위치, 카트의 속도, 막대의 각도, 막대의 각속도)

In [ ]:
# DQN algorithm for CartPole 동작 과정 요약

for n_epi in range(10000):
    
    while not done: # 한 episode 동안 ..
        # 각각의 state(길이 4인 벡터)의 정보를 네트워크에 전달 -> action을 선택(0 or 1)
        # action을 선택하고, 그 action을 통해 다음 state(s_prime)와 reward를 받음
        # 해당 transition을 ReplayBuffer에 put()

    if memory.size() > 2000:
        # episode가 끝날때마다 train을 실시
        # 즉, ReplayBuffer에서 mini_batch를 뽑아 해당 데이터에 대한 Loss를 계산 -> 그라디언트를 통해 Qnet에 대한 weight를 업데이트
        # 이때 Q-target network를 이용하여 정답지를 계산(학습 대상이 아님)
    
    if n_epi % print_interval == 0 and n_epi!=0:
        # 20 episode마다 가장 최근 20개 episode의 보상 총합의 평균을 프린트
        # q 네트워크의 파라미터를 q_target 네트워크로 복사

env.close()

##### 라이브러리 import

In [1]:
import gym # OpenAI GYM 라이브러리
import collections # 리플레이 버퍼를 구현할때 사용 (deque : first in first out)
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

##### 하이퍼 파라미터 정의

In [32]:
# Hyperparameters
learning_rate = 0.0005
gamma = 0.98
buffer_limit = 50000
batch_size = 32 # 하나의 미니 배치 안에 32개의 데이터가 쓰인다는 의미 -> Loss function(L(seta))를 정의할때 기댓값 연산자 반드시 필요(같은 상태 s에서 같은 액션 a를 취하더라도 매번 다른 상태에 도달할 수 있기 때문에) -> 이때 사용되는 데이터의 개수가 mini batch size

##### 리플레이 버퍼 클래스

In [33]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen = buffer_limit) # buffer_limit = 50,000

    def put(self, transition): # 데이터를 buffer에 넣어주는 함수
        self.buffer.append(transition)

    def sample(self, n): # buffer에서 랜덤하게 batch_size만큼의 데이터를 뽑아주는 함수
        mini_batch = random.sample(self.buffer, n) # buffer에서 랜덤하게 n개의 데이터를 뽑아서 mini_batch에 저장(32개의 transition 데이터)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch: # transition example : [ 0.11703863  1.3383894  -0.12113763 -2.0172946 ]
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a]) # a는 스칼라 값이므로 []로 감싸줌
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype = torch.float), torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype = torch.float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

##### Q밸류 네트워크 클래스

In [34]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon): # obs : observation으로 보임(current state)
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon: # initial epsilon = 0.08
            return random.randint(0, 1) # random하게 action 선택(exploration)
        else:
            return out.argmax().item() # Q value가 높은 action 선택(exploitation) : return값은 0 or 1

##### 학습 함수

In [35]:
def train(q, q_target, memory, optimizer): # episode 하나가 끝날 때마다 train을 실시, 한 번 호출될 때마다 10개의 mini batch를 뽑아 총 10번 update (총 320개 data)
    for i in range(10):
        s, a, r, s_prime, done_mask = memory.sample(batch_size)

        q_out = q(s) # q 네트워크에 state를 넣어서 q value를 구함, shape : (32, 2), first value : [0.1336,  0.1596]
        q_a = q_out.gather(1, a) # q value 중에서 실제 취한 action에 해당하는 q value만 골라냄, shape : (32, 1), first value : [0.1336], gather(1, a)는 dim = 1(열 방향)에서 a에 해당하는 Index를 골라내는 작업
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) # q_target 네트워크에 s_prime을 넣어서 나온 q value 중에서 최대값을 골라냄, shape : (32, 1), first value : [0.1177]
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)

        optimizer.zero_grad()
        loss.backward() # Loss function을 미분해서 각 파라미터에 대한 gradient를 계산
        optimizer.step() # 계산된 gradient를 통해 파라미터를 업데이트

##### 메인 함수

In [44]:
import time

def main():
    env = gym.make('CartPole-v1')
    q = Qnet() # 학습 네트워크
    q_target = Qnet() # 타깃 네트워크
    q_target.load_state_dict(q.state_dict()) # q 네트워크의 파라미터를 q_target 네트워크로 복사하는 과정 (이러한 update과정은 20 episode마다 실시됨)
    memory = ReplayBuffer()

    br = True
    print_interval = 20
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr = learning_rate) # update과정에서는 q 네트워크의 파라미터만 사용됨 (off-policy)

    for n_epi in range(10000):
        start_time = time.time()
        epsilon = max(0.01, 0.08 - 0.01 * (n_epi / 200)) # Linear annealing from 8% to 1% (decaying epsilon greedy)
        s, _ = env.reset()
        # print("s : ", s)
        done = False

        while not done: # 한 episode 동안
            if time.time() - start_time > 600 : # 강제 종료 조건 삽입
                br = False
                break
            
            a = q.sample_action(torch.from_numpy(s).float(), epsilon) # a = 0 or 1
            s_prime, r, done, truncated, info = env.step(a) 
            done_mask = 0.0 if done else 1.0
            memory.put((s, a, r / 100.0, s_prime, done_mask)) # transition을 통해 얻은 데이터를 ReplayBuffer에 저장 / reward scaling -> why?
            s = s_prime # state update
            score += r # reward 누적 : 최근 20 episode마다 평균 reward를 프린트하기 위함(개선 여부 확인을 위해)
            if done:
                break
        if br == False:
            break

        if memory.size() > 2000: # ReplayBuffer에 데이터가 충분히 쌓이지 않았을 때 학습을 진행하면 초기의 데이터가 많이 재사용되어 학습이 치우칠 수 있기 때문
            train(q, q_target, memory, optimizer)

        if n_epi % print_interval == 0 and n_epi != 0:
            q_target.load_state_dict(q.state_dict()) # 20 episode마다 q 네트워크의 파라미터를 q_target 네트워크로 복사(20 episode동안 파라미터를 얼려둠)
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score / print_interval, memory.size(), epsilon * 100))
            score = 0.0
    env.close()

if __name__ == '__main__':
    main()


n_episode :20, score : 10.3, n_buffer : 207, eps : 7.9%
n_episode :40, score : 9.6, n_buffer : 399, eps : 7.8%
n_episode :60, score : 9.6, n_buffer : 591, eps : 7.7%
n_episode :80, score : 9.6, n_buffer : 782, eps : 7.6%
n_episode :100, score : 10.0, n_buffer : 982, eps : 7.5%
n_episode :120, score : 9.9, n_buffer : 1180, eps : 7.4%
n_episode :140, score : 10.4, n_buffer : 1389, eps : 7.3%
n_episode :160, score : 9.7, n_buffer : 1583, eps : 7.2%
n_episode :180, score : 10.2, n_buffer : 1786, eps : 7.1%
n_episode :200, score : 10.1, n_buffer : 1988, eps : 7.0%
n_episode :220, score : 10.0, n_buffer : 2188, eps : 6.9%
n_episode :240, score : 9.9, n_buffer : 2386, eps : 6.8%
n_episode :260, score : 9.9, n_buffer : 2584, eps : 6.7%
n_episode :280, score : 10.3, n_buffer : 2790, eps : 6.6%
n_episode :300, score : 11.8, n_buffer : 3025, eps : 6.5%
n_episode :320, score : 20.0, n_buffer : 3425, eps : 6.4%
n_episode :340, score : 74.5, n_buffer : 4915, eps : 6.3%
n_episode :360, score : 21.3, 

<복습>

Q. 본 과정은 On-policy인가 Off-policy인가? 왜 그렇게 생각하는가? (hint : Q-learning)

Q. 결국 본 과정을 통해 얻은 결과물은 무엇인가? (hint : value network)

Q. 다음은 TD-target을 이용해 파라미터 업데이트를 실시한다. 그렇다면 파라미터는 언제 업데이트되는가?
